In [2]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from typing import List
from langchain_core.documents import Document
import os
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

pdf_loader = PyPDFLoader("docs/Lessentiel-des-regles-de-golf-2019.pdf")
documents = pdf_loader.load()

splits =  text_splitter.split_documents(documents)

print(f"Split the documents into {len(splits)} chunks.")

Split the documents into 54 chunks.


In [28]:
splits[0]

Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 13.0 (Windows)', 'creationdate': '2018-11-28T15:37:22+01:00', 'moddate': '2018-11-28T15:37:31+01:00', 'trapped': '/False', 'source': 'docs/Lessentiel-des-regles-de-golf-2019.pdf', 'total_pages': 28, 'page': 0, 'page_label': '1'}, page_content='www.ffgolf.org\nL ’ESSENTIEL \nDES RÈGLES \nDE GOLF\nApplicables en janvier 2019')

In [29]:
splits[0].metadata

{'producer': 'Adobe PDF Library 15.0',
 'creator': 'Adobe InDesign CC 13.0 (Windows)',
 'creationdate': '2018-11-28T15:37:22+01:00',
 'moddate': '2018-11-28T15:37:31+01:00',
 'trapped': '/False',
 'source': 'docs/Lessentiel-des-regles-de-golf-2019.pdf',
 'total_pages': 28,
 'page': 0,
 'page_label': '1'}

In [30]:
documents[0]

Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 13.0 (Windows)', 'creationdate': '2018-11-28T15:37:22+01:00', 'moddate': '2018-11-28T15:37:31+01:00', 'trapped': '/False', 'source': 'docs/Lessentiel-des-regles-de-golf-2019.pdf', 'total_pages': 28, 'page': 0, 'page_label': '1'}, page_content='www.ffgolf.org\nL ’ESSENTIEL \nDES RÈGLES \nDE GOLF\nApplicables en janvier 2019')

In [33]:
splits[2].page_content[:1000]  # Display the first 1000 characters of the third chunk

'Un glossaire des termes apparaissant en bleu dans le texte \nest disponible à la fin de ce document. \nUn golf peut définir des Règles locales spécifiques à son parcours, \nelles sont généralement disponibles sur la carte de score.'

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from typing import List
import os

def load_documents_from_folder(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
            # Fusionner toutes les pages d'un PDF en un seul document
            pdf_documents = loader.load()
            merged_content = "\n".join([doc.page_content for doc in pdf_documents])
            merged_document = Document(page_content=merged_content, metadata=pdf_documents[0].metadata)
            documents.append(merged_document)
        else:
            print(f"extension non supportée: {filename}")
            continue
    return documents

# Charger les documents depuis le dossier "docs"
folder_path = "docs"
documents = load_documents_from_folder(folder_path)

print(f"Chargement de {len(documents)} documents")
splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")

Chargement de 4 documents
Split the documents into 3284 chunks.


In [4]:
from langchain_openai import OpenAIEmbeddings

# Fonction pour diviser les chunks en lots
def batch_chunks(chunks, batch_size):
    for i in range(0, len(chunks), batch_size):
        yield chunks[i:i + batch_size]

# Créer les embeddings par lots
embedding = OpenAIEmbeddings()
batch_size = 100  # Ajustez cette valeur en fonction de la taille moyenne des chunks
document_embeddings = []

for batch in batch_chunks(splits, batch_size):
    batch_embeddings = embedding.embed_documents([chunk.page_content for chunk in batch])
    document_embeddings.extend(batch_embeddings)

print(f"Created embeddings for {len(document_embeddings)} document chunks.")

Created embeddings for 3284 document chunks.


In [38]:
document_embeddings[0]

[0.0036552399396896362,
 -0.032694268971681595,
 0.004792068153619766,
 -0.027567273005843163,
 -0.03756362944841385,
 0.022298574447631836,
 -0.0033106491900980473,
 0.009925506077706814,
 -0.016501717269420624,
 -0.017918726429343224,
 0.003368617733940482,
 -0.022762322798371315,
 -0.02159006893634796,
 -0.00955193117260933,
 -0.01141336653381586,
 -0.005668037571012974,
 0.03246239572763443,
 0.00028138907509855926,
 0.025312939658761024,
 0.014170093461871147,
 0.0002089283661916852,
 0.018008900806307793,
 -0.01258561946451664,
 -0.01910386234521866,
 -0.005384635645896196,
 -1.2108233931940049e-05,
 0.0012898005079478025,
 -0.010479427874088287,
 -0.000820416200440377,
 -0.01780278980731964,
 0.01818924769759178,
 0.007967457175254822,
 -0.0029064794071018696,
 -0.012456800788640976,
 -0.015290819108486176,
 -0.0008184034377336502,
 -0.010563160292804241,
 -0.006125344894826412,
 0.009043095633387566,
 -0.0025860420428216457,
 0.02621467225253582,
 -0.007832197472453117,
 -0.005

In [ ]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

# Ajoutez votre jeton Hugging Face ici
hf_token = os.getenv("HF_TOKEN")

embedding_function = SentenceTransformerEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs={"use_auth_token": hf_token}
)

document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])
print(document_embeddings[0])

C:\Users\Marcel\AppData\Local\Temp\ipykernel_24244\1401922850.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(
c:\Users\Marcel\Documents\Rag_reglement_golf\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Marcel\Documents\Rag_reglement_golf\.venv\Lib\site-packages\sentence_transformers\SentenceTransformer.py:204: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransf

[-0.021963248029351234, 0.05450787395238876, -0.05467120185494423, -0.03536735847592354, 0.04996240884065628, 0.08946152776479721, -0.012241081334650517, 0.013433041982352734, 0.010079753585159779, 0.09041000157594681, -0.04588855430483818, 0.0003029831568710506, -0.06913859397172928, 0.04818253591656685, 0.009264728985726833, -0.029074881225824356, -0.055304188281297684, 0.005027757957577705, -0.07612117379903793, 0.02615479752421379, 0.06915108859539032, -0.01700880378484726, -0.005050709471106529, 0.018641097471117973, -0.010753283277153969, -0.06548301875591278, -0.023809852078557014, 0.09551383554935455, -0.031920626759529114, -0.033652711659669876, 0.011608312837779522, -0.025544732809066772, 0.09347377717494965, -0.010992352850735188, 0.0359448678791523, 0.020057152956724167, 0.07560458779335022, -0.14268793165683746, 0.015569405630230904, -0.0028354907408356667, -0.0763612762093544, -0.041548289358615875, -0.011152345687150955, 0.03583404794335365, -0.05921400338411331, 0.02034

In [6]:
from langchain_chroma import Chroma

embedding_function =  OpenAIEmbeddings()
collection_name = "my_collection"


# Fonction pour diviser les documents en lots
def batch_documents(documents, batch_size):
    for i in range(0, len(documents), batch_size):
        yield documents[i:i + batch_size]

# Taille du lot (ajustez en fonction de la taille moyenne des documents)
batch_size = 100
vectorstore = None

# Traiter les documents par lots
for batch in batch_documents(splits, batch_size):
    if vectorstore is None:
        # Créer le vectorstore pour le premier lot
        vectorstore = Chroma.from_documents(
            collection_name=collection_name,
            documents=batch,
            embedding=embedding_function,
            persist_directory="chroma_db"
        )
    else:
        # Ajouter les documents au vectorstore existant
        vectorstore.add_documents(batch)

print("Vectorstore créé avec succès.")

Vectorstore créé avec succès.


In [9]:
# perform similarity search

query = "balle dans l'eau ?"
search_results = vectorstore.similarity_search(query, k=3)

print(f"\nTop 3 résultats de recherche similaires pour la requête : '{query}'")
for i, result in enumerate(search_results, 1):
    print(f"Result{i}")
    print(f"Source : {result.metadata.get('source', 'N/A')}")
    print(f"Contenu : {result.page_content[:200]}") 
    print()


Top 3 résultats de recherche similaires pour la requête : 'balle dans l'eau ?'
Result1
Source : docs\Decisions-2016-2017.pdf
Contenu : 14-6/1 
Balle se déplaçant dans l'eau d'un obstacle d'eau après que le stance a été pris  
Q. Une balle est au repos dans un obstacle d'eau peu profond où le courant est rapide. 
Le joueur entre prude

Result2
Source : docs\Decisions-2016-2017.pdf
Contenu : décision ? 
R. La balle d'origine est perdue et l'autre balle est en jeu avec une pénalité de coup et 
distance – voir Règle 27-1a et Définition de "Balle perdue". 
 
Décision connexe : 
• 15/11 Mauva

Result3
Source : docs\Decisions-2016-2017.pdf
Contenu : l'obstacle d'eau 
1-4/7 
Balle provisoire  
 Jouée avec le seul motif que la 
balle d'origine puisse être dans un 
obstacle d'eau 
27-2a/2 
 Jouée en croyant que la balle 
d'origine peut être perdue e



In [10]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
retriever.invoke("balle dans l'eau ?")

[Document(id='f124cd52-d270-4bef-b6e8-cc0503c7db90', metadata={'producer': 'Adobe PDF Library 11.0', 'creator': 'Acrobat PDFMaker 11 pour Word', 'page': 0, 'title': '', 'page_label': '1', 'moddate': '2015-12-22T22:56:32+01:00', 'source': 'docs\\Decisions-2016-2017.pdf', 'total_pages': 738, 'company': 'FFGolf', 'author': 'JBDB', 'creationdate': '2015-12-09T19:45:17+01:00'}, page_content="14-6/1 \nBalle se déplaçant dans l'eau d'un obstacle d'eau après que le stance a été pris  \nQ. Une balle est au repos dans un obstacle d'eau peu profond où le courant est rapide. \nLe joueur entre prudemment dans l'eau, marche vers la balle et prend son stance . \nLa balle se déplace alors, probablement en raison du courant. Quelle est la \ndécision ? \nR. Quand une balle est dans l'eau et qu'il n'est pas évident que les actions du joueur \naient provoqué le déplacement de la balle, le bénéfice du doute devrait être accordé \net aucune pénalité ne devrait être appliquée. Cependant, si les actions du jo

In [11]:
from langchain_core.prompts import ChatPromptTemplate

template = """répond à la question en vous basant uniquement sur le contexte suivant :
{context}

Question : {question}

Réponse : """

prompt = ChatPromptTemplate.from_template(template)


In [29]:
from langchain_openai import ChatOpenAI
import os 
import langchain 

llm = ChatOpenAI(model="gpt-4o-mini")

# os.environ["OPENAI_API_KEY"]= os.getenv("OPENAI_API_KEY")# Configuration de l'environnement pour activer le tracing V2 de LangChain
# os.environ["LANGCHAIN_TRACING_V2"] = "true"

# # Configuration de l'environnement avec la clé API de LangChain
# os.environ["LANGCHAIN_API_KEY"] = "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"

# # Configuration de l'environnement pour définir le projet LangChain comme "rag_Golf"
# os.environ["LANCHAIN_PROJECT"] = "rag_Golf"

LANGSMITH_TRACING="true"
LANGSMITH_ENDPOINT="https://api.smith.langchain.com"
LANGSMITH_API_KEY="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"
LANGSMITH_PROJECT="rag_Golf"
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")




from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
    {"context": retriever, "question":RunnablePassthrough()} | prompt
)

rag_chain.invoke("balle dans l'eau ?")

ChatPromptValue(messages=[HumanMessage(content='répond à la question en vous basant uniquement sur le contexte suivant :\n[Document(id=\'f124cd52-d270-4bef-b6e8-cc0503c7db90\', metadata={\'creationdate\': \'2015-12-09T19:45:17+01:00\', \'creator\': \'Acrobat PDFMaker 11 pour Word\', \'source\': \'docs\\\\Decisions-2016-2017.pdf\', \'author\': \'JBDB\', \'moddate\': \'2015-12-22T22:56:32+01:00\', \'title\': \'\', \'page\': 0, \'company\': \'FFGolf\', \'page_label\': \'1\', \'total_pages\': 738, \'producer\': \'Adobe PDF Library 11.0\'}, page_content="14-6/1 \\nBalle se déplaçant dans l\'eau d\'un obstacle d\'eau après que le stance a été pris  \\nQ. Une balle est au repos dans un obstacle d\'eau peu profond où le courant est rapide. \\nLe joueur entre prudemment dans l\'eau, marche vers la balle et prend son stance . \\nLa balle se déplace alors, probablement en raison du courant. Quelle est la \\ndécision ? \\nR. Quand une balle est dans l\'eau et qu\'il n\'est pas évident que les acti

In [ ]:
def docs2string(docs):
    return "\n\n".join([doc.page_content for doc in docs])


In [16]:
rag_chain = (
    {"context": retriever | docs2string,"question":RunnablePassthrough()} | prompt
)

rag_chain.invoke("balle dans l'eau ?")

ChatPromptValue(messages=[HumanMessage(content='répond à la question en vous basant uniquement sur le contexte suivant :\n14-6/1 \nBalle se déplaçant dans l\'eau d\'un obstacle d\'eau après que le stance a été pris  \nQ. Une balle est au repos dans un obstacle d\'eau peu profond où le courant est rapide. \nLe joueur entre prudemment dans l\'eau, marche vers la balle et prend son stance . \nLa balle se déplace alors, probablement en raison du courant. Quelle est la \ndécision ? \nR. Quand une balle est dans l\'eau et qu\'il n\'est pas évident que les actions du joueur \naient provoqué le déplacement de la balle, le bénéfice du doute devrait être accordé \net aucune pénalité ne devrait être appliquée. Cependant, si les actions du joueur ont \nclairement provoqué le déplacement de la balle, il devrait être pénalisé d\'un coup \nselon la Règle 18 -2 et devrait replacer la balle. Par exemple, si un joueur saute \ndans l\'eau près de la balle et ce faisant crée une gerbe qui déplace la balle

In [20]:
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

rag_chain = (
    {"context": retriever | docs2string, "question": RunnablePassthrough()} 
    | prompt
    | llm
    | StrOutputParser()
)

question = "ma balle est tombée dans l'eau"
reponse = rag_chain.invoke(question)
print(f"Réponse: {reponse}")

Réponse: Si votre balle est tombée dans l'eau, vous devez d'abord déterminer si elle est dans un obstacle d'eau ou dans de l'eau fortuite. Selon la Règle de l'obstacle d'eau, vous avez la possibilité de droppper votre balle dans l'obstacle d'eau, ou jouer une balle provisoire si vous pensez que votre balle pourrait être dans l'obstacle. Si votre balle est retrouvée dans l'obstacle, vous pouvez la jouer comme seconde balle. Si elle est retrouvée hors de l'obstacle, alors vous devez agir selon la Règle 26-1/3.


In [ ]:
# Conversion du RAG

In [21]:
# Example de conversion du RAG

from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=reponse)
])

In [22]:
chat_history

[HumanMessage(content="ma balle est tombée dans l'eau", additional_kwargs={}, response_metadata={}),
 AIMessage(content="Si votre balle est tombée dans l'eau, vous devez d'abord déterminer si elle est dans un obstacle d'eau ou dans de l'eau fortuite. Selon la Règle de l'obstacle d'eau, vous avez la possibilité de droppper votre balle dans l'obstacle d'eau, ou jouer une balle provisoire si vous pensez que votre balle pourrait être dans l'obstacle. Si votre balle est retrouvée dans l'obstacle, vous pouvez la jouer comme seconde balle. Si elle est retrouvée hors de l'obstacle, alors vous devez agir selon la Règle 26-1/3.", additional_kwargs={}, response_metadata={})]

In [24]:
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt =(
    "À partir de l'historique de discussion et de la dernière question de l'utilisateur,"
    "qui pourrait faire référence au contexte de l'historique de discussion,"
    "formulez une question autonome et compréhensible."
    "Sans l'historique de discussion, ne répondez pas à la question."
    "Reformulez-la simplement si nécessaire, sinon renvoyez-la telle quelle."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
[
        ("system",contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}")
    ]
)

contextualize_chain =  contextualize_q_prompt | llm | StrOutputParser()
contextualize_chain.invoke({"input": "Ma balle est dans le lac ", "chat_history": chat_history})



"Avez-vous besoin de conseils sur la façon de récupérer votre balle ou sur les règles à suivre si elle est dans un lac lors d'une partie de golf ?"

In [25]:
from langchain.chains import create_history_aware_retriever

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

history_aware_retriever.invoke({"input": "Ma balle est dans le lac ", "chat_history": chat_history})

[Document(id='506e5d68-63ae-4459-bf6e-73df0ad42268', metadata={'page': 0, 'moddate': '2018-11-28T15:37:31+01:00', 'creationdate': '2018-11-28T15:37:22+01:00', 'producer': 'Adobe PDF Library 15.0', 'page_label': '1', 'creator': 'Adobe InDesign CC 13.0 (Windows)', 'trapped': '/False', 'source': 'docs\\Lessentiel-des-regles-de-golf-2019.pdf', 'total_pages': 28}, page_content='votre balle était sur le green et se déplace après que vous l’avez déjà relevée et \nreplacée, la balle doit être replacée à son emplacement d’origine.\nSi votre balle au repos est relevée ou déplacée par toute personne ou toute influence \nextérieure avant que le coup ne soit joué, votre balle doit être replacée à son empla- \ncement d’origine.\nVous devriez faire attention lorsque vous êtes près d’une balle au repos. Si vous causez \nle déplacement de votre propre balle ou de celle de votre adversaire, vous encourrez \nnormalement une pénalité (sauf sur le green).\nJOUER VOTRE BALLE COMME ELLE REPOSE RÈGLE 9\nAvant

In [31]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "Vous êtes un assistant IA qui répond aux questions en se basant sur le contexte fourni."),
    ("system","Context : {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human","{input}")
])

question_answering_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answering_chain)

In [28]:
rag_chain.invoke({"input": "Ma balle est dans le lac ", "chat_history": chat_history})

{'input': 'Ma balle est dans le lac ',
 'chat_history': [HumanMessage(content="ma balle est tombée dans l'eau", additional_kwargs={}, response_metadata={}),
  AIMessage(content="Si votre balle est tombée dans l'eau, vous devez d'abord déterminer si elle est dans un obstacle d'eau ou dans de l'eau fortuite. Selon la Règle de l'obstacle d'eau, vous avez la possibilité de droppper votre balle dans l'obstacle d'eau, ou jouer une balle provisoire si vous pensez que votre balle pourrait être dans l'obstacle. Si votre balle est retrouvée dans l'obstacle, vous pouvez la jouer comme seconde balle. Si elle est retrouvée hors de l'obstacle, alors vous devez agir selon la Règle 26-1/3.", additional_kwargs={}, response_metadata={})],
 'context': [Document(id='a43810a4-4111-4dfb-bef1-4c21bb6424bc', metadata={'source': 'docs\\Lessentiel-des-regles-de-golf-2019.pdf', 'total_pages': 28, 'creator': 'Adobe InDesign CC 13.0 (Windows)', 'page': 0, 'moddate': '2018-11-28T15:37:31+01:00', 'trapped': '/False'